# Preserving Lifestyle as an Expat on Other Continent
#  - Moving from Toronto, Canada to Bucharest, Romania –

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Prepare the Environment</a>

2. <a href="#item1">Download and Explore Dataset</a>

3. <a href="#item2">Explore Neighborhoods in Toronto - Canada</a>

4. <a href="#item3">Analyze Each Neighborhood in Toronto - Canada</a>

5. <a href="#item4">Cluster Neighborhoods in Toronto - Canada</a>

6. <a href="#item5">Examine Clusters and Labels in Toronto - Canada</a>   

7. <a href="#item1">Download and Explore Dataset</a>

8. <a href="#item2">Explore Neighborhoods in Bucharest - Romania</a>

9. <a href="#item3">Analyze Each Neighborhood in Bucharest - Romania/a>

10. <a href="#item4">Cluster Neighborhoods in Bucharest - Romania</a>

11. <a href="#item5">Examine Clusters and Labels in Bucharest - Romania</a>   
</font>
</div>

## 1. Prepare the Environment

In [1]:
!conda install -c conda-forge beautifulsoup4 --yes

!conda install -c conda-forge geopy --yes

!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    beautifulsoup4-4.8.0       |           py36_0         144 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following packages will be UPDATED:

    beautifulsoup4:  4.7.1-py36_1      --> 4.8.0-py36_0         conda-forge
    ca-certificates: 2019.5.15-1       --> 2019.9.11-hecc5488_0 conda-forge
    certifi:         2019.6.16-py36_1  --> 2019.9.11-py36_0     conda-forg

In [4]:
#Import core libraries
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from bs4 import BeautifulSoup # library to parse HTML and XML documents
import matplotlib.cm as cm #For ploting
import matplotlib.colors as colors
from sklearn.cluster import KMeans# import k-means from clustering stage
import folium # map rendering library

## 2. Download and Explore Dataset

Download the Toronto data from Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [5]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_url, 'lxml')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className=document.documentElement.className.replace(/(^|\s)client-nojs(\s|$)/,"$1client-js$2");RLCONF={"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":906439794,"wgRevisionId":906439794,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["CS1 errors: deprecated parameters","Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":!1,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","Feb

### Create dataframe

In [6]:
table = soup.find_all('table', class_ = "wikitable sortable")

In [7]:
len(table)

1

In [8]:
table = table[0]
type(table)

bs4.element.Tag

#### Tranform the data into a pandas dataframe

In [9]:
table_rows = table.find_all('tr')

res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)
df = pd.DataFrame(res, columns=["Postcode", "Borough", "Neighbourhood"])
df=df.rename(columns = {'Postcode':'Postal Code', 'Neighbourhood':'Neighborhood'})
df.head(10)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [10]:
df.shape

(288, 3)

In [11]:
df1 = df[df.Borough != 'Not assigned']
df1.head(10)

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [12]:
df1.shape

(211, 3)

In [13]:
df1.Neighborhood[df1.Neighborhood == 'Not assigned'] = df1.Borough
df1.head(10)

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/generic.py:8682: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/opt/conda/envs/Python36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3267: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [14]:
df2 = df1.groupby(['Postal Code','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df2.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [15]:
df2.shape

(103, 3)

#### Get geo data (latitude, longitude)

In [17]:
url="http://cocl.us/Geospatial_data"
df3=pd.read_csv(url)
df3.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
result = pd.merge(df2,df3, on='Postal Code')
result.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [19]:
print('boroughs = {} , neighborhoods = {}'.format(
        len(result['Borough'].unique()),
        result.shape[0]
    )
)

boroughs = 11 , neighborhoods = 103


In [20]:
result.shape

(103, 5)

#### Use geopy library to get the latitude and longitude values of Toronto

In [21]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="Clustering")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Canada is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Canada is 43.653963, -79.387207.


## 3. Explore Neighborhoods in Toronto - Canada

### Map Toronto - Canada

#### Create a map of Toronto with neighborhoods superimposed on top

In [26]:
map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(result['Latitude'], result['Longitude'], result['Borough'], result['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map) 
map

#### Define Foursquare credentials and version

In [27]:
CLIENT_ID = '0IO3DZ4VAS2IN0X1AG23CW0RHLMSDQLKJBB4FORBJGOAS1FD'
CLIENT_SECRET = 'OSUFNRO3QGE0PRNHWCUWETYW4ZJI0KTBCJA2ORHNLO4FRCVC'
VERSION = '20180604'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0IO3DZ4VAS2IN0X1AG23CW0RHLMSDQLKJBB4FORBJGOAS1FD
CLIENT_SECRET:OSUFNRO3QGE0PRNHWCUWETYW4ZJI0KTBCJA2ORHNLO4FRCVC


#### Get the top 200 venues that are in Toronto within a radius of 500 meters

In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=200):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [29]:
# type your answer here
toronto_venues = getNearbyVenues(names=result['Neighborhood'],
                                   latitudes=result['Latitude'],
                                   longitudes=result['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

#### Check the size of the resulting dataframe

In [30]:
print(toronto_venues.shape)
toronto_venues.head()

(2255, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


In [31]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,5,5,5,5,5,5
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",2,2,2,2,2,2
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",9,9,9,9,9,9
"Alderwood, Long Branch",9,9,9,9,9,9
"Bathurst Manor, Downsview North, Wilson Heights",20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",21,21,21,21,21,21
Berczy Park,57,57,57,57,57,57


#### Find out how many unique categories can be curated from all the returned venues

In [32]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 274 uniques categories.


In [33]:
toronto_venues.shape

(2255, 7)

## 4. Analyze each Neighborhood in Toronto - Canada

In [34]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [35]:
toronto_onehot.shape

(2255, 274)

#### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [36]:
toronto_onehot = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_onehot.head()

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [37]:
toronto_onehot.shape

(100, 274)

#### Identify and eliminate neigborhoods withouth venues from dataframe

In [38]:
result.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [39]:
result.shape

(103, 5)

In [40]:
result2=result
result2.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [41]:
result2['Neighborhood'].isin(toronto_onehot['Neighborhood']).value_counts()
# 3 neighborhoods without venues

True     100
False      3
Name: Neighborhood, dtype: int64

In [42]:
set(result2['Neighborhood']).intersection(set(toronto_onehot['Neighborhood']))

{'Adelaide, King, Richmond',
 'Agincourt',
 "Agincourt North, L'Amoreaux East, Milliken, Steeles East",
 'Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown',
 'Alderwood, Long Branch',
 'Bathurst Manor, Downsview North, Wilson Heights',
 'Bayview Village',
 'Bedford Park, Lawrence Manor East',
 'Berczy Park',
 'Birch Cliff, Cliffside West',
 'Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe',
 'Brockton, Exhibition Place, Parkdale Village',
 'Business Reply Mail Processing Centre 969 Eastern',
 'CFB Toronto, Downsview East',
 'CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara',
 'Cabbagetown, St. James Town',
 'Caledonia-Fairbanks',
 'Canada Post Gateway Processing Centre',
 'Cedarbrae',
 'Central Bay Street',
 'Chinatown, Grange Park, Kensington Market',
 'Christie',
 'Church and Wellesley',
 'Clairlea, Golden Mile, Oakridge',
 "Clarks Corners, Sullivan, Ta

In [43]:
result3=result2.drop([16, 21, 93], axis=0)
result3.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [44]:
result3.shape

(100, 5)

#### Check index of geo dataframe and venues dataframe

In [45]:
result3['Neighborhood'].isin(toronto_onehot['Neighborhood']).value_counts()

True    100
Name: Neighborhood, dtype: int64

#### Print each neighborhood along with the top 5 most common venues

In [46]:
num_top_venues = 5

for hood in toronto_onehot['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_onehot[toronto_onehot['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
         venue  freq
0  Coffee Shop  0.08
1         Café  0.05
2          Bar  0.04
3   Steakhouse  0.04
4        Hotel  0.03


----Agincourt----
                venue  freq
0        Skating Rink   0.2
1      Breakfast Spot   0.2
2  Chinese Restaurant   0.2
3      Sandwich Place   0.2
4              Lounge   0.2


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                             venue  freq
0                       Playground   0.5
1                             Park   0.5
2                    Metro Station   0.0
3  Molecular Gastronomy Restaurant   0.0
4       Modern European Restaurant   0.0


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                 venue  freq
0        Grocery Store  0.22
1             Pharmacy  0.11
2           Beer Store  0.11
3  Fried Chicken Joint  0.11
4       Sandwich Place  0.11


----Alderwood, Long Branch----
            v

#### Put venues output into a pandas dataframe

In [47]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Create the new dataframe and display the top 10 venues for each neighborhood

In [48]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_onehot['Neighborhood']

for ind in np.arange(toronto_onehot.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_onehot.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,Steakhouse,Burger Joint,Cosmetics Shop,Hotel,American Restaurant,Restaurant,Thai Restaurant
1,Agincourt,Lounge,Sandwich Place,Breakfast Spot,Skating Rink,Chinese Restaurant,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Playground,Park,Women's Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Fried Chicken Joint,Pharmacy,Pizza Place,Fast Food Restaurant,Coffee Shop,Beer Store,Sandwich Place,Women's Store,Dog Run
4,"Alderwood, Long Branch",Pizza Place,Pub,Coffee Shop,Gym,Pharmacy,Sandwich Place,Skating Rink,Pool,Women's Store,Department Store


## 5.Cluster Neighborhoods in Toronto - Canada

Run *k*-means to cluster the neighborhood into 5 clusters.

In [49]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_onehot.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 0, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [50]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = result3

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,1,Fast Food Restaurant,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Harbor / Marina
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,4,Bar,Women's Store,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Fast Food Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1,Medical Center,Electronics Store,Pizza Place,Intersection,Breakfast Spot,Rental Car Location,Mexican Restaurant,Doner Restaurant,Diner,Discount Store
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1,Coffee Shop,Korean Restaurant,Women's Store,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1,Hakka Restaurant,Bank,Athletics & Sports,Bakery,Thai Restaurant,Caribbean Restaurant,Fried Chicken Joint,Eastern European Restaurant,Dumpling Restaurant,Drugstore


In [51]:
toronto_merged.shape

(100, 16)

#### Visualize Toronto clusters

In [52]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Cluster 1

In [53]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,"Agincourt North, L'Amoreaux East, Milliken, St...",Playground,Park,Women's Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
20,"Silver Hills, York Mills",Park,Women's Store,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
23,York Mills West,Park,Bank,Convenience Store,Women's Store,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
25,Parkwoods,Park,Food & Drink Shop,Women's Store,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
30,"CFB Toronto, Downsview East",Park,Airport,Women's Store,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
40,East Toronto,Park,Intersection,Coffee Shop,Convenience Store,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
50,Rosedale,Park,Trail,Playground,Building,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
74,Caledonia-Fairbanks,Park,Women's Store,Market,Fast Food Restaurant,Comic Shop,Concert Hall,Event Space,Ethiopian Restaurant,Comfort Food Restaurant,Empanada Restaurant
98,Weston,Park,Convenience Store,Women's Store,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore


### Cluster 2

In [54]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Rouge, Malvern",Fast Food Restaurant,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Harbor / Marina
2,"Guildwood, Morningside, West Hill",Medical Center,Electronics Store,Pizza Place,Intersection,Breakfast Spot,Rental Car Location,Mexican Restaurant,Doner Restaurant,Diner,Discount Store
3,Woburn,Coffee Shop,Korean Restaurant,Women's Store,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
4,Cedarbrae,Hakka Restaurant,Bank,Athletics & Sports,Bakery,Thai Restaurant,Caribbean Restaurant,Fried Chicken Joint,Eastern European Restaurant,Dumpling Restaurant,Drugstore
5,Scarborough Village,Playground,Construction & Landscaping,Women's Store,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
6,"East Birchmount Park, Ionview, Kennedy Park",Department Store,Discount Store,Coffee Shop,Hobby Shop,Convenience Store,Dumpling Restaurant,Diner,Dog Run,Doner Restaurant,Donut Shop
7,"Clairlea, Golden Mile, Oakridge",Bakery,Park,Intersection,Fast Food Restaurant,Metro Station,Bus Line,Bus Station,Soccer Field,Costume Shop,Construction & Landscaping
8,"Cliffcrest, Cliffside, Scarborough Village West",Motel,American Restaurant,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
9,"Birch Cliff, Cliffside West",Café,General Entertainment,Skating Rink,College Stadium,Concert Hall,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
10,"Dorset Park, Scarborough Town Centre, Wexford ...",Indian Restaurant,Pet Store,Latin American Restaurant,Vietnamese Restaurant,Chinese Restaurant,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store


### Cluster 3

In [55]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Lawrence Park,Park,Bus Line,Swim School,Women's Store,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
100,"Kingsview Village, Martin Grove Gardens, Richv...",Mobile Phone Shop,Park,Pizza Place,Bus Line,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant


### Cluster 4

In [56]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
94,"Cloverdale, Islington, Martin Grove, Princess ...",Bank,Women's Store,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Fast Food Restaurant


### Cluster 5

In [57]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Highland Creek, Rouge Hill, Port Union",Bar,Women's Store,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Fast Food Restaurant


## 6. Examine Clusters and Labels in Toronto - Canada

In [59]:
print ("Cluster 1 labels = Park, Stores")
print ("Cluster 2 labels = Restaurants, Shops")
print ("Cluster 3 labels = Park, Bus Line, Shops")
print ("Cluster 4 labels = Banks, Women Store, Drugstore")
print ("Cluster 5 labels = Bars, Women Stores, Diner")

Cluster 1 labels = Park, Stores
Cluster 2 labels = Restaurants, Shops
Cluster 3 labels = Park, Bus Line, Shops
Cluster 4 labels = Banks, Women Store, Drugstore
Cluster 5 labels = Bars, Women Stores, Diner


# Clustering Neiborhoods in Bucharest (Romania)

## 7. Download and Explore Dataset

In [60]:
#Get Neighborhoods List for Bucharest
#url='https://ro.wikipedia.org/wiki/Lista_cartierelor_din_București'

In [61]:
website_text = requests.get('https://ro.wikipedia.org/wiki/Lista_cartierelor_din_București').text
soup = BeautifulSoup(website_text,'xml')
print(soup.prettify())

<?xml version="1.0" encoding="utf-8"?>
<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="ro">
 <head>
  <meta charset="UTF-8"/>
  <title>
   Lista cartierelor din București - Wikipedia
  </title>
  <script>
   document.documentElement.className=document.documentElement.className.replace(/(^|\s)client-nojs(\s|$)/,"$1client-js$2");RLCONF={"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Lista_cartierelor_din_București","wgTitle":"Lista cartierelor din București","wgCurRevisionId":12939177,"wgRevisionId":12939177,"wgArticleId":201242,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Liste legate de București","Cartiere din București"],"wgBreakFrames":!1,"wgPageContentLanguage":"ro","wgPageContentModel":"wikitext","wgSeparatorTransformTable":[",\t.",".\t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","ianuarie","februarie","martie","aprilie","ma

### Clear the Data

In [62]:
#replacing Romanian special text characters
def remove_special_characters(value):

    value = value.replace('ș', 's')
    value = value.replace('ț', 't')
    value = value.replace('ă', 'a')
    value = value.replace('â', 'a')
    value = value.replace('Ș', 's')

    return value

#get all lines that contain neiborhoods names
table = soup.find_all('a')
all_links = soup.find_all("a")[4:]
bucuresti_neighborhoods = []
#get only string data, for neiborhoods list only and remove special characters
for l in all_links:
    link = l.string
    if link is not None:
        if 'Jump to' not in link:
            link = remove_special_characters(link)
            bucuresti_neighborhoods.append(link)
            if 'Vitan'in link:
                break

print(bucuresti_neighborhoods)

['13 Septembrie', 'Aviatorilor', 'Aviatiei', 'Balta Alba', 'Baicului', 'Berceni', 'Brancusi', 'Bucurestii Noi', 'Baneasa', 'Centrul Civic', 'Centrul istoric', 'Chitila', 'Colentina', 'Cotroceni', 'Crangasi', 'Dealul Spirii', 'Diham', 'Doamna Ghica', 'Domenii', 'Dorobanti', 'Dristor', 'Drumul Taberei', 'Dudesti', 'Damaroaia', 'Ferentari', 'Floreasca', 'Fundeni', 'Gara de Nord', 'Ghencea', 'Giulesti', 'Giurgiului', 'Grozavesti', 'Grivita', 'Iancului', 'Ion Creanga', 'Lipscani', 'Militari', 'Mosilor', 'Morarilor', 'Mihai Bravu', 'Muncii', 'Obor', 'Ozana', 'Oltenitei', 'Pajura', 'Pantelimon', 'Piata Romana', 'Piata Universitatii', 'Piata Resita', 'Pipera', 'Primaverii', 'Rahova', 'Salajan', 'Straulesti', 'stefan cel Mare', 'Tei', 'Timpuri Noi', 'Tineretului', 'Titan', 'Unirii', 'Vatra Luminoasa', 'Vacaresti', 'Vergului', 'Voluntari', 'Victoriei', 'Vitan']


In [63]:
###df_neiborhoods = pd.DataFrame(list(zip(bucaresti_nh)), columns =['Neighboorhod'])
###print(df_neiborhoods.head())

### Get Bucharest Neighborhoods Locations using Geolocator

In [64]:
lat = []
long = []

for neighborhoods in bucuresti_neighborhoods:
    address = neighborhoods + ', Bucuresti'
    geolocator = Nominatim(user_agent="Clustering")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinate is {}, {}.'.format(latitude, longitude))
    lat.append(latitude)
    long.append(longitude)
    
bucuresti_locations = pd.DataFrame(list(zip(bucuresti_neighborhoods, lat, long)), columns =['Neighborhood','Latitude', 'Longitude'])
print(bucuresti_locations.head())

The geograpical coordinate is 44.4215188, 26.0651147604058.
The geograpical coordinate is 44.45950405, 26.0810384064127.
The geograpical coordinate is 44.4824568, 26.0939991835941.
The geograpical coordinate is 44.417151, 26.1818769257132.
The geograpical coordinate is 44.4442326, 26.1433029.
The geograpical coordinate is 44.3118585, 26.2013948182611.
The geograpical coordinate is 44.4196415, 26.0077920896181.
The geograpical coordinate is 44.4902616, 26.0323010885202.
The geograpical coordinate is 44.5132121, 26.0664273910951.
The geograpical coordinate is 44.42664105, 26.1081296075433.
The geograpical coordinate is 44.4326488, 26.1031787331426.
The geograpical coordinate is 44.5059871, 25.987703.
The geograpical coordinate is 44.46642565, 26.1510161022924.
The geograpical coordinate is 44.43276835, 26.0718705342058.
The geograpical coordinate is 44.4531845, 26.0486868666958.
The geograpical coordinate is 44.4228885, 26.0693905.
The geograpical coordinate is 44.439484, 26.1569639.
The

In [65]:
bucuresti_locations.shape

(66, 3)

In [66]:
bucuresti_locations

,Neighborhood,Latitude,Longitude
0,13 Septembrie,44.421519,26.065115
1,Aviatorilor,44.459504,26.081038
2,Aviatiei,44.482457,26.093999
3,Balta Alba,44.417151,26.181877
4,Baicului,44.444233,26.143303
5,Berceni,44.311858,26.201395
6,Brancusi,44.419641,26.007792
7,Bucurestii Noi,44.490262,26.032301
8,Baneasa,44.513212,26.066427
9,Centrul Civic,44.426641,26.108130


#### Use geopy library to get the latitude and longitude values of Bucharest (Romania)

In [67]:
address = 'Bucharest, Romania'

geolocator = Nominatim(user_agent="Clustering")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bucharest, Romania is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bucharest, Romania is 44.4361414, 26.1027202.


## 8. Explore Neighborhoods in Bucharest - Romania

### Map Bucharest

In [68]:
map = folium.Map(location=[latitude, longitude], zoom_start=12)
# add markers to map
for lat, lng, neighborhood in zip(bucuresti_locations['Latitude'], bucuresti_locations['Longitude'], bucuresti_locations['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map) 
map

#### Define Foursquare credentials and version again

In [69]:
CLIENT_ID = '0IO3DZ4VAS2IN0X1AG23CW0RHLMSDQLKJBB4FORBJGOAS1FD'
CLIENT_SECRET = 'OSUFNRO3QGE0PRNHWCUWETYW4ZJI0KTBCJA2ORHNLO4FRCVC'
VERSION = '20180604'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0IO3DZ4VAS2IN0X1AG23CW0RHLMSDQLKJBB4FORBJGOAS1FD
CLIENT_SECRET:OSUFNRO3QGE0PRNHWCUWETYW4ZJI0KTBCJA2ORHNLO4FRCVC


#### Get the top 200 venues that are in Bucharest within a radius of 500 meters

In [70]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=200):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [71]:
# type your answer here
bucharest_venues = getNearbyVenues(names=bucuresti_locations['Neighborhood'],
                                   latitudes=bucuresti_locations['Latitude'],
                                   longitudes=bucuresti_locations['Longitude']
                                  )

13 Septembrie
Aviatorilor
Aviatiei
Balta Alba
Baicului
Berceni
Brancusi
Bucurestii Noi
Baneasa
Centrul Civic
Centrul istoric
Chitila
Colentina
Cotroceni
Crangasi
Dealul Spirii
Diham
Doamna Ghica
Domenii
Dorobanti
Dristor
Drumul Taberei
Dudesti
Damaroaia
Ferentari
Floreasca
Fundeni
Gara de Nord
Ghencea
Giulesti
Giurgiului
Grozavesti
Grivita
Iancului
Ion Creanga
Lipscani
Militari
Mosilor
Morarilor
Mihai Bravu
Muncii
Obor
Ozana
Oltenitei
Pajura
Pantelimon
Piata Romana
Piata Universitatii
Piata Resita
Pipera
Primaverii
Rahova
Salajan
Straulesti
stefan cel Mare
Tei
Timpuri Noi
Tineretului
Titan
Unirii
Vatra Luminoasa
Vacaresti
Vergului
Voluntari
Victoriei
Vitan


#### Check the size of the resulting dataframe

In [72]:
print(bucharest_venues.shape)
bucharest_venues.head()

(1395, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,13 Septembrie,44.421519,26.065115,ABBATE Salon,44.420100,26.063916,Salon / Barbershop
1,13 Septembrie,44.421519,26.065115,Prosper Fitness,44.420153,26.065467,Gym / Fitness Center
2,13 Septembrie,44.421519,26.065115,Green Day Spa,44.425332,26.062242,Cosmetics Shop
3,13 Septembrie,44.421519,26.065115,The Roof,44.420061,26.066116,Lounge
4,13 Septembrie,44.421519,26.065115,LIDL,44.424773,26.063779,Supermarket


In [73]:
bucharest_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
13 Septembrie,10,10,10,10,10,10
Aviatiei,19,19,19,19,19,19
Aviatorilor,31,31,31,31,31,31
Baicului,12,12,12,12,12,12
Balta Alba,16,16,16,16,16,16
Baneasa,4,4,4,4,4,4
Brancusi,8,8,8,8,8,8
Bucurestii Noi,8,8,8,8,8,8
Centrul Civic,66,66,66,66,66,66


#### Find out how many unique categories can be curated from all the returned venues

In [74]:
print('There are {} uniques categories.'.format(len(bucharest_venues['Venue Category'].unique())))

There are 191 uniques categories.


## 9. Analyze Each Neighborhood in Bucharest - Romania

In [75]:
# one hot encoding
bucharest_onehot = pd.get_dummies(bucharest_venues[['Venue Category']], prefix="", prefix_sep="")

In [76]:
# add neighborhood column back to dataframe
bucharest_onehot['Neighborhood'] = bucharest_venues['Neighborhood'] 

In [77]:
# move neighborhood column to the first column
fixed_columns = [bucharest_onehot.columns[-1]] + list(bucharest_onehot.columns[:-1])
bucharest_onehot = bucharest_onehot[fixed_columns]

In [78]:
bucharest_onehot.head()

,Neighborhood,Accessories Store,Arcade,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Betting Shop,Bike Shop,Bistro,Boarding House,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Burger Joint,Bus Station,Bus Stop,Café,Camera Store,Casino,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Currency Exchange,Dance Studio,Department Store,Dessert Shop,Diner,Dog Run,Doner Restaurant,Drugstore,Eastern European Restaurant,Electronics Store,Event Space,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Food Court,Football Stadium,Forest,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Health & Beauty Service,Herbs & Spices Store,Historic Site,History Museum,Hookah Bar,Hospital,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Indie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Leather Goods Store,Lebanese Restaurant,Light Rail Station,Lounge,Market,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Mini Golf,Mobile Phone Shop,Modern European Restaurant,Monastery,Multiplex,Music Store,Music Venue,Nightclub,Noodle House,Outdoor Sculpture,Palace,Paper / Office Supplies Store,Park,Pedestrian Plaza,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pizza Place,Platform,Plaza,Pool,Pool Hall,Pub,Rental Car Location,Rest Area,Restaurant,Rock Club,Romanian Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Science Museum,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Swiss Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Tennis Stadium,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Train Station,Tunnel,Turkish Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Wine Bar,Wine Shop,Women's Store
0,13 Septembrie,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,13 Septembrie,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,13 Septembrie,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,13 Septembrie,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

#### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [79]:
bucharest_onehot = bucharest_onehot.groupby('Neighborhood').mean().reset_index()
bucharest_onehot.head()

,Neighborhood,Accessories Store,Arcade,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Betting Shop,Bike Shop,Bistro,Boarding House,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Burger Joint,Bus Station,Bus Stop,Café,Camera Store,Casino,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Currency Exchange,Dance Studio,Department Store,Dessert Shop,Diner,Dog Run,Doner Restaurant,Drugstore,Eastern European Restaurant,Electronics Store,Event Space,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Food Court,Football Stadium,Forest,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Health & Beauty Service,Herbs & Spices Store,Historic Site,History Museum,Hookah Bar,Hospital,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Indie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Leather Goods Store,Lebanese Restaurant,Light Rail Station,Lounge,Market,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Mini Golf,Mobile Phone Shop,Modern European Restaurant,Monastery,Multiplex,Music Store,Music Venue,Nightclub,Noodle House,Outdoor Sculpture,Palace,Paper / Office Supplies Store,Park,Pedestrian Plaza,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pizza Place,Platform,Plaza,Pool,Pool Hall,Pub,Rental Car Location,Rest Area,Restaurant,Rock Club,Romanian Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Science Museum,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Swiss Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Tennis Stadium,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Train Station,Tunnel,Turkish Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Wine Bar,Wine Shop,Women's Store
0,13 Septembrie,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.100000,0.0,0.0,0.0,0.0,0.0000,0.000000,0.0000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.100000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.1,0.0,0.000000,0.0,0.000000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.100000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.200000,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.200000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Aviatiei,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.157895,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0000,0.052632,0.0000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [80]:
bucharest_onehot.shape

(64, 192)

In [81]:
bucuresti_locations.shape

(66, 3)

#### Identify and eliminate neigborhoods withouth venues from dataframe

In [82]:
bucuresti_locations['Neighborhood'].isin(bucharest_onehot['Neighborhood']).value_counts()
# 2 neighborhoods without venues

True     64
False     2
Name: Neighborhood, dtype: int64

In [83]:
set(bucuresti_locations['Neighborhood']).intersection(set(bucharest_onehot['Neighborhood']))

{'13 Septembrie',
 'Aviatiei',
 'Aviatorilor',
 'Baicului',
 'Balta Alba',
 'Baneasa',
 'Brancusi',
 'Bucurestii Noi',
 'Centrul Civic',
 'Centrul istoric',
 'Chitila',
 'Colentina',
 'Cotroceni',
 'Crangasi',
 'Damaroaia',
 'Dealul Spirii',
 'Diham',
 'Doamna Ghica',
 'Domenii',
 'Dorobanti',
 'Dristor',
 'Drumul Taberei',
 'Dudesti',
 'Ferentari',
 'Floreasca',
 'Gara de Nord',
 'Ghencea',
 'Giulesti',
 'Giurgiului',
 'Grivita',
 'Grozavesti',
 'Iancului',
 'Ion Creanga',
 'Lipscani',
 'Mihai Bravu',
 'Militari',
 'Morarilor',
 'Mosilor',
 'Muncii',
 'Obor',
 'Oltenitei',
 'Ozana',
 'Pajura',
 'Pantelimon',
 'Piata Resita',
 'Piata Romana',
 'Piata Universitatii',
 'Pipera',
 'Primaverii',
 'Rahova',
 'Salajan',
 'Straulesti',
 'Tei',
 'Timpuri Noi',
 'Tineretului',
 'Titan',
 'Unirii',
 'Vacaresti',
 'Vatra Luminoasa',
 'Vergului',
 'Victoriei',
 'Vitan',
 'Voluntari',
 'stefan cel Mare'}

In [84]:
bucuresti_locations2=bucuresti_locations.drop([5, 26], axis=0)
bucuresti_locations2.shape

(64, 3)

#### Check index of geo dataframe and venues dataframe

In [85]:
bucuresti_locations2['Neighborhood'].isin(bucharest_onehot['Neighborhood']).value_counts()
# 2 neighborhoods without venues

True    64
Name: Neighborhood, dtype: int64

#### Print each neighborhood along with the top 5 most common venues

In [86]:
num_top_venues = 5

for hood in bucharest_onehot['Neighborhood']:
    print("----"+hood+"----")
    temp = bucharest_onehot[bucharest_onehot['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----13 Septembrie----
                venue  freq
0                 Bar   0.2
1         Supermarket   0.2
2  Salon / Barbershop   0.2
3      Cosmetics Shop   0.1
4         Pizza Place   0.1


----Aviatiei----
           venue  freq
0          Hotel  0.16
1         Bakery  0.16
2     Restaurant  0.05
3    Pizza Place  0.05
4  Grocery Store  0.05


----Aviatorilor----
                       venue  freq
0         Italian Restaurant  0.10
1                     Bakery  0.10
2                 Restaurant  0.06
3  Middle Eastern Restaurant  0.06
4                Supermarket  0.06


----Baicului----
                  venue  freq
0           Pizza Place  0.17
1                   Gym  0.17
2  Fast Food Restaurant  0.17
3    Italian Restaurant  0.08
4           Bus Station  0.08


----Balta Alba----
               venue  freq
0  Mobile Phone Shop  0.12
1              Diner  0.06
2     Sandwich Place  0.06
3               Park  0.06
4      Bowling Alley  0.06


----Baneasa----
                  ven

#### Put venues output into a pandas dataframe

In [87]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Create the new dataframe and display the top 10 venues for each neighborhood

In [88]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods2_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods2_venues_sorted['Neighborhood'] = bucharest_onehot['Neighborhood']

for ind in np.arange(bucharest_onehot.shape[0]):
    neighborhoods2_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bucharest_onehot.iloc[ind, :], num_top_venues)

neighborhoods2_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,13 Septembrie,Supermarket,Salon / Barbershop,Bar,Cosmetics Shop,Pizza Place,Gym / Fitness Center,Lounge,Event Space,Football Stadium,Food Court
1,Aviatiei,Hotel,Bakery,Supermarket,Italian Restaurant,Lebanese Restaurant,Light Rail Station,Dessert Shop,Grocery Store,Gym,Restaurant
2,Aviatorilor,Italian Restaurant,Bakery,Mediterranean Restaurant,Middle Eastern Restaurant,Supermarket,Romanian Restaurant,Restaurant,Hotel,Cosmetics Shop,Tea Room
3,Baicului,Pizza Place,Fast Food Restaurant,Gym,Bus Station,Italian Restaurant,Supermarket,Café,Grocery Store,Park,Women's Store
4,Balta Alba,Mobile Phone Shop,Sporting Goods Shop,Grocery Store,Bookstore,Café,Farmers Market,Shoe Store,Sandwich Place,Jewelry Store,Park


## 10. Cluster Neighborhoods in Bucharest - Romania

Run *k*-means to cluster the neighborhood into 5 clusters.

In [89]:
# set number of clusters
kclusters = 5

bucharest_grouped_clustering = bucharest_onehot.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bucharest_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 2, 2, 0, 2, 2, 1, 2, 2, 2], dtype=int32)

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [90]:
# add clustering labels
neighborhoods2_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

bucharest_merged = bucuresti_locations2

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
bucharest_merged = bucharest_merged.join(neighborhoods2_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

bucharest_merged.head()

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,13 Septembrie,44.421519,26.065115,1,Supermarket,Salon / Barbershop,Bar,Cosmetics Shop,Pizza Place,Gym / Fitness Center,Lounge,Event Space,Football Stadium,Food Court
1,Aviatorilor,44.459504,26.081038,2,Italian Restaurant,Bakery,Mediterranean Restaurant,Middle Eastern Restaurant,Supermarket,Romanian Restaurant,Restaurant,Hotel,Cosmetics Shop,Tea Room
2,Aviatiei,44.482457,26.093999,2,Hotel,Bakery,Supermarket,Italian Restaurant,Lebanese Restaurant,Light Rail Station,Dessert Shop,Grocery Store,Gym,Restaurant
3,Balta Alba,44.417151,26.181877,2,Mobile Phone Shop,Sporting Goods Shop,Grocery Store,Bookstore,Café,Farmers Market,Shoe Store,Sandwich Place,Jewelry Store,Park
4,Baicului,44.444233,26.143303,0,Pizza Place,Fast Food Restaurant,Gym,Bus Station,Italian Restaurant,Supermarket,Café,Grocery Store,Park,Women's Store


In [91]:
bucharest_merged.shape

(64, 14)

### Visualize Bucharest Clusters

In [92]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bucharest_merged['Latitude'], bucharest_merged['Longitude'], bucharest_merged['Neighborhood'], bucharest_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Cluster 1

In [98]:
bucharest_merged.loc[bucharest_merged['Cluster Labels'] == 0, bucharest_merged.columns[[0] + list(range(4, bucharest_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Baicului,Pizza Place,Fast Food Restaurant,Gym,Bus Station,Italian Restaurant,Supermarket,Café,Grocery Store,Park,Women's Store
28,Ghencea,Athletics & Sports,Bus Station,Pizza Place,Gym,Stadium,Supermarket,Salon / Barbershop,Theme Restaurant,Romanian Restaurant,Bar
45,Pantelimon,Pizza Place,Market,Women's Store,Event Space,Forest,Football Stadium,Food Court,Food & Drink Shop,Fast Food Restaurant,Farmers Market
52,Salajan,Farmers Market,Gym / Fitness Center,Pool,Bus Station,Pizza Place,Athletics & Sports,Soccer Field,Gym,Eastern European Restaurant,Women's Store
61,Vacaresti,Pool,Italian Restaurant,Basketball Court,Tennis Stadium,Soccer Field,Cocktail Bar,Theme Park Ride / Attraction,Romanian Restaurant,Garden,Gym


### Cluster 2

In [99]:
bucharest_merged.loc[bucharest_merged['Cluster Labels'] == 1, bucharest_merged.columns[[0] + list(range(4, bucharest_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,13 Septembrie,Supermarket,Salon / Barbershop,Bar,Cosmetics Shop,Pizza Place,Gym / Fitness Center,Lounge,Event Space,Football Stadium,Food Court
6,Brancusi,Supermarket,Park,Pizza Place,Coffee Shop,Café,Bed & Breakfast,Women's Store,Forest,Football Stadium,Food Court
11,Chitila,Supermarket,Pub,Pizza Place,Train Station,Women's Store,Electronics Store,Football Stadium,Food Court,Food & Drink Shop,Fast Food Restaurant
15,Dealul Spirii,Supermarket,Indian Restaurant,Gym / Fitness Center,Plaza,Pizza Place,Coffee Shop,Eastern European Restaurant,Lounge,Bar,Pub
23,Damaroaia,Supermarket,Concert Hall,Women's Store,Event Space,Fountain,Forest,Football Stadium,Food Court,Food & Drink Shop,Fast Food Restaurant
30,Giurgiului,Supermarket,Mobile Phone Shop,Sporting Goods Shop,Bus Station,Jewelry Store,Light Rail Station,Bakery,Farmers Market,Falafel Restaurant,Fountain
36,Militari,Supermarket,Falafel Restaurant,Restaurant,Italian Restaurant,Bus Station,Pharmacy,Women's Store,Event Space,Football Stadium,Food Court
44,Pajura,Restaurant,Supermarket,Department Store,Bus Stop,Women's Store,Eye Doctor,Forest,Football Stadium,Food Court,Food & Drink Shop
48,Piata Resita,Supermarket,Farmers Market,Park,Basketball Court,Women's Store,Eye Doctor,Forest,Football Stadium,Food Court,Food & Drink Shop
51,Rahova,Light Rail Station,Supermarket,Grocery Store,Event Space,Forest,Football Stadium,Food Court,Food & Drink Shop,Fast Food Restaurant,Farmers Market


### Cluster 3

In [100]:
bucharest_merged.loc[bucharest_merged['Cluster Labels'] == 2, bucharest_merged.columns[[0] + list(range(4, bucharest_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Aviatorilor,Italian Restaurant,Bakery,Mediterranean Restaurant,Middle Eastern Restaurant,Supermarket,Romanian Restaurant,Restaurant,Hotel,Cosmetics Shop,Tea Room
2,Aviatiei,Hotel,Bakery,Supermarket,Italian Restaurant,Lebanese Restaurant,Light Rail Station,Dessert Shop,Grocery Store,Gym,Restaurant
3,Balta Alba,Mobile Phone Shop,Sporting Goods Shop,Grocery Store,Bookstore,Café,Farmers Market,Shoe Store,Sandwich Place,Jewelry Store,Park
7,Bucurestii Noi,Shop & Service,Dessert Shop,Metro Station,Pharmacy,Supermarket,Gym,Bakery,Theater,Women's Store,Football Stadium
8,Baneasa,Gym,Forest,Spa,Gym / Fitness Center,Electronics Store,Football Stadium,Food Court,Food & Drink Shop,Fast Food Restaurant,Farmers Market
9,Centrul Civic,Café,Clothing Store,Coffee Shop,Hotel,Italian Restaurant,Bakery,Burger Joint,Gym / Fitness Center,Pizza Place,Plaza
10,Centrul istoric,Coffee Shop,Pub,Café,Historic Site,Pizza Place,Hotel,Romanian Restaurant,Hookah Bar,Bookstore,Palace
12,Colentina,Rental Car Location,Supermarket,Food & Drink Shop,Fast Food Restaurant,Park,Eastern European Restaurant,Cosmetics Shop,Spa,Hotel,Auto Garage
13,Cotroceni,Italian Restaurant,Bistro,Café,Park,Electronics Store,Pedestrian Plaza,Tennis Court,Market,Bar,Jazz Club
14,Crangasi,Cupcake Shop,Pizza Place,Supermarket,Tennis Court,Plaza,Electronics Store,Sandwich Place,Falafel Restaurant,Shoe Store,Farmers Market


### Cluster 4

In [101]:
bucharest_merged.loc[bucharest_merged['Cluster Labels'] == 3, bucharest_merged.columns[[0] + list(range(4, bucharest_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
39,Mihai Bravu,Gift Shop,Women's Store,Fried Chicken Joint,Fountain,Forest,Football Stadium,Food Court,Food & Drink Shop,Fast Food Restaurant,Farmers Market


### Cluster 5

In [102]:
bucharest_merged.loc[bucharest_merged['Cluster Labels'] == 4, bucharest_merged.columns[[0] + list(range(5, bucharest_merged.shape[1]))]]

,Neighborhood,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,Ferentari,Auto Garage,Women's Store,Event Space,Fountain,Forest,Football Stadium,Food Court,Food & Drink Shop,Fast Food Restaurant


## 11. Examine Clusters and Labels in Bucharest - Romania

In [103]:
print ("Cluster 1 labels = Sports")
print ("Cluster 2 labels = Supermarkets, Pharmacyes, Food")
print ("Cluster 3 labels = High-life")
print ("Cluster 4 labels = Tranzit zone")
print ("Cluster 5 labels = Auto, Food")

Cluster 1 labels = Sports
Cluster 2 labels = Supermarkets, Pharmacyes, Food
Cluster 3 labels = High-life
Cluster 4 labels = Tranzit zone
Cluster 5 labels = Auto, Food


*Thank you!*